<a href="https://colab.research.google.com/github/jespinozallanos/ia4business/blob/master/Part%201%20-%20Optimizing%20Business%20Processes/qlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inteligencia Artificial Aplicada a Negocios y Empresas
# Parte 1 - Optimizacion de los flujos de trabajo en un almacen con Q-Learning


## Importacion de las librerias


In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
import numpy as np

R = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Libro2.csv",header=None )
R=np.array(R)
R

array([[0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
       [1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1],
       [0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0],
       [1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1],
       [1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0]])

In [2]:
import numpy as np

## Configuracion de los parametros gamma y alpha para el algoritmo de Q-Learning


In [80]:
gamma = 0.2
alpha = 0.5

# PARTE 1 - DEFINICION DEL ENTORNO


## Definicion de los estados


In [82]:
location_to_state = {'A': 0,
                     'B': 1,
                     'C': 2,
                     'D': 3,
                     'E': 4,
                     'F': 5,
                     'G': 6, 
                     'H': 7, 
                     'I': 8,
                     'J': 9,
                     'K': 10,
                     'L': 11}

## Definicion de las acciones

In [81]:
actions = [0, 1, 2, 3, 4, 5, 6, 7,8,9,10,11]

## Definicion de las recompensas
## Columnas:    A,B,C,D,E,F,G,H,I,J,K,L

In [11]:
R = np.array([[0,1,0,0,0,0,0,0,0,0,0,0], # A
              [1,0,1,0,0,1,0,0,0,0,0,0], # B
              [0,1,0,0,0,0,1,0,0,0,0,0], # C
              [0,0,0,0,0,0,0,1,0,0,0,0], # D
              [0,0,0,0,0,0,0,0,1,0,0,0], # E
              [0,1,0,0,0,0,0,0,0,1,0,0], # F
              [0,0,1,0,0,0,1,1,0,0,0,0], # G
              [0,0,0,1,0,0,1,0,0,0,0,1], # H
              [0,0,0,0,1,0,0,0,0,1,0,0], # I
              [0,0,0,0,0,1,0,0,1,0,1,0], # J
              [0,0,0,0,0,0,0,0,0,1,0,1], # K
              [0,0,0,0,0,0,0,1,0,0,1,0]])# L


              

# PARTE 2 - CONSTRUCCION DE LA SOLUCION DE IA CON Q-LEARNING


## Transformacion inversa de estados a ubicaciones

In [83]:
state_to_location = {state : location for location, state in location_to_state.items()}

## Crear la funcion final que nos devuelva la ruta óptima

In [84]:
def route(starting_location, ending_location):
    R_new = np.copy(R)
    ending_state = location_to_state[ending_location]
    R_new[ending_state, ending_state] = 1000
    
    Q = np.array(np.zeros([12, 12]))
    for i in range(1000):
        current_state = np.random.randint(0, 12)
        playable_actions = []
        for j in range(12):
            if R_new[current_state, j] > 0:
                playable_actions.append(j)
        next_state = np.random.choice(playable_actions)
        TD = R_new[current_state, next_state] + gamma*Q[next_state, np.argmax(Q[next_state,])] - Q[current_state, next_state]
        Q[current_state, next_state] = Q[current_state, next_state] + alpha*TD
    
    route = [starting_location]
    next_location = starting_location
    while(next_location != ending_location):
        starting_state = location_to_state[starting_location]
        next_state = np.argmax(Q[starting_state, ])
        next_location = state_to_location[next_state]
        route.append(next_location)
        starting_location = next_location
    return route

# PARTE 3 - PONER EL MODELO EN PRODUCCION


In [88]:
def best_route(starting_location, intermediary_location, ending_location):
    return route(starting_location, intermediary_location) + route(intermediary_location, ending_location)[1:]

# Imprimir la ruta final
print("Ruta Elegida:")
print(best_route('L','G', 'F'))

Ruta Elegida:
['L', 'E', 'G', 'F']
